In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avazu-ctr-prediction/sampleSubmission.gz
/kaggle/input/avazu-ctr-prediction/train.gz
/kaggle/input/avazu-ctr-prediction/test.gz


In [2]:
import gzip
import shutil

input_path = '/kaggle/input/avazu-ctr-prediction/train.gz'
output_path = '/kaggle/working/train.csv'

with gzip.open(input_path, 'rb') as f_in:
    with open(output_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Extraction complete! File saved as:", output_path)


Extraction complete! File saved as: /kaggle/working/train.csv


In [3]:
n_rows = 300000

# Load 300,000 rows from the compressed train.gz file
df = pd.read_csv("/kaggle/working/train.csv", nrows=n_rows)

# Preview the data
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [4]:
Y = df['click'].values

In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [6]:
X = df.drop(['click', 'id', 'hour', 'device_id', 'device_ip', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model'], axis=1).values
# X = df.drop(columns=["id", "click"])

In [7]:
enc = OneHotEncoder(handle_unknown= 'ignore')

In [9]:
# 80/20 split for training and testing

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:

enc_X_train =  enc.fit_transform(X_train)
print(enc_X_train[0])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12 stored elements and shape (1, 901)>
  Coords	Values
  (0, 2)	1.0
  (0, 7)	1.0
  (0, 13)	1.0
  (0, 16)	1.0
  (0, 257)	1.0
  (0, 522)	1.0
  (0, 526)	1.0
  (0, 602)	1.0
  (0, 675)	1.0
  (0, 680)	1.0
  (0, 719)	1.0
  (0, 899)	1.0


In [11]:
enc_X_test = enc.transform(X_test)

In [12]:
decision_tree = DecisionTreeClassifier(criterion='gini', min_samples_split=30)
parameters = {'max_depth': [3, 10, None]}

In [13]:
grid_search = GridSearchCV(decision_tree, parameters, n_jobs=-1, cv=3, scoring='roc_auc')

In [14]:
grid_search.fit(X_train, Y_train)


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(min_samples_split=30),
             n_jobs=-1, param_grid={'max_depth': [3, 10, None]},
             scoring='roc_auc')

In [15]:
print(grid_search.best_params_)

{'max_depth': 10}


In [16]:
decision_tree_best = grid_search.best_estimator_
prob = decision_tree_best.predict_proba(X_test)[:, 1]

In [17]:
from sklearn.metrics import roc_auc_score, log_loss
auc = roc_auc_score(Y_test, prob)
loss = log_loss(Y_test, prob)

print(f"AUC Score: {auc:.4f}")
print(f"Log Loss: {loss:.4f}")

AUC Score: 0.7159
Log Loss: 0.4202
